In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_rose_ext/train'
test_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_rose_ext/test'
validation_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_rose_ext/val'

# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Load base models without the top layers
input_tensor = Input(shape=(224, 224, 3))
vgg19_base = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor)
densenet_base = DenseNet121(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze base model layers
for layer in vgg19_base.layers:
    layer.trainable = False
for layer in densenet_base.layers:
    layer.trainable = False

# Extract features
vgg19_output = GlobalAveragePooling2D()(vgg19_base.output)
densenet_output = GlobalAveragePooling2D()(densenet_base.output)

# Concatenate features
merged = Concatenate()([vgg19_output, densenet_output])
merged = Dense(1024, activation='relu')(merged)
predictions = Dense(1, activation='sigmoid')(merged)

# Define final model
model = Model(inputs=input_tensor, outputs=predictions)
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=15
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


Found 2238 images belonging to 2 classes.
Found 482 images belonging to 2 classes.
Found 480 images belonging to 2 classes.
Epoch 1/15


C:\Users\HP\Documents\anaconda2\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


69/69 ━━━━━━━━━━━━━━━━━━━━ 679s 10s/step - accuracy: 0.7691 - loss: 0.4965 - val_accuracy: 0.9396 - val_loss: 0.2164
Epoch 2/15
 1/69 ━━━━━━━━━━━━━━━━━━━━ 8:16 7s/step - accuracy: 0.9688 - loss: 0.1800

C:\Users\HP\Documents\anaconda2\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


69/69 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.9688 - loss: 0.1800 - val_accuracy: 0.9458 - val_loss: 0.2107
Epoch 3/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 621s 9s/step - accuracy: 0.9690 - loss: 0.1715 - val_accuracy: 0.9500 - val_loss: 0.1479
Epoch 4/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 119s 2s/step - accuracy: 0.9375 - loss: 0.1688 - val_accuracy: 0.9542 - val_loss: 0.1468
Epoch 5/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 626s 9s/step - accuracy: 0.9720 - loss: 0.1216 - val_accuracy: 0.9604 - val_loss: 0.1157
Epoch 6/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 126s 2s/step - accuracy: 1.0000 - loss: 0.0823 - val_accuracy: 0.9604 - val_loss: 0.1157
Epoch 7/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 642s 9s/step - accuracy: 0.9806 - loss: 0.0877 - val_accuracy: 0.9604 - val_loss: 0.1051
Epoch 8/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 120s 2s/step - accuracy: 1.0000 - loss: 0.0475 - val_accuracy: 0.9625 - val_loss: 0.1024
Epoch 9/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 641s 9s/step - accuracy: 0.9855 - loss: 0.0678 - val_accuracy: 0.9521 - val_loss: 0.

In [2]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


16/16 ━━━━━━━━━━━━━━━━━━━━ 118s 7s/step - accuracy: 0.9768 - loss: 0.0656
Test Loss: 0.07274282723665237
Test Accuracy: 0.9708333611488342
Validation Loss: 0.07453273981809616
Validation Accuracy: 0.9688796401023865
